## Deribit API Request
#### ALL Options data 

### Specfic Settlement Period

In [19]:
import pandas as pd
import json
import requests
from tqdm import tqdm
import concurrent.futures

# functions
def get_option_name_and_settlement(coin):
    """
    :param coin: crypto-currency coin name ('BTC', 'ETH')
    :return: 2 lists:
                        1.  list of traded options for the selected coin;
                        2.  list of settlement period for the selected coin.
    """
    r = requests.get("https://test.deribit.com/api/v2/public/get_instruments?currency=" + coin + "&kind=option")
    result = json.loads(r.text)

    # get option name
    name = pd.json_normalize(result['result'])['instrument_name']
    name = list(name)

    # get option settlement period
    settlement_period = pd.json_normalize(result['result'])['settlement_period']
    settlement_period = list(settlement_period)

    return name, settlement_period


def fetch_option_data(option_name):
    """Fetch the option data for a given option name."""
    r = requests.get(f'https://test.deribit.com/api/v2/public/get_order_book?instrument_name={option_name}')
    result = json.loads(r.text)
    return pd.json_normalize(result['result'])


def get_option_data(coin):
    """
    :param coin: crypto-currency coin name ('BTC', 'ETH')
    :return: pandas data frame with all option data for a given coin, filtered for options with settlement in months
    """
    # get option name and settlement
    coin_name, settlement_period = get_option_name_and_settlement(coin)

    # filter for options that have settlement period in 'day'
    coin_name_filtered = [coin_name[i] for i in range(len(coin_name)) if settlement_period[i] == 'month']

    # initialize progress bar
    pbar = tqdm(total=len(coin_name_filtered))

    # Fetch data concurrently using ThreadPoolExecutor
    with concurrent.futures.ThreadPoolExecutor() as executor:
        future_to_option = {executor.submit(fetch_option_data, name): name for name in coin_name_filtered}
        coin_df = []
        
        for future in concurrent.futures.as_completed(future_to_option):
            try:
                data = future.result()
                data['settlement_period'] = 'month'
                coin_df.append(data)
            except Exception as exc:
                print(f'Error fetching data: {exc}')
            pbar.update(1)

    # finalize data frame
    if len(coin_df) > 0:
        coin_df = pd.concat(coin_df)

    # remove useless columns from coin_df
    columns = ['state', 'estimated_delivery_price']
    if not coin_df.empty:
        coin_df.drop(columns, inplace=True, axis=1)

    # close the progress bar
    pbar.close()

    return coin_df

# download data -- BTC and ETH Options
btc_data = get_option_data('BTC')


C:\Users\Ethan Vaz Falcao\AppData\Local\Temp\ipykernel_24420\2227378591.py:66: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  coin_df = pd.concat(coin_df)
100%|██████████| 780/780 [01:45<00:00,  7.37it/s]


In [20]:
btc_data

,best_bid_amount,best_ask_amount,underlying_index,underlying_price,ask_iv,bid_iv,mark_iv,interest_rate,best_bid_price,best_ask_price,...,greeks.theta,greeks.vega,greeks.gamma,greeks.delta,stats.volume_usd,stats.volume,stats.price_change,stats.low,stats.high,settlement_period
0,0.0,0.0,BTC-25OCT24,67154.81,0.00,0.00,171.48,0.0,0.0000,0.0000,...,-0.00002,0.00000,0.00000,0.00000,0.00,0.0,NaN,NaN,NaN,month
0,0.1,35.6,BTC-25OCT24,67154.81,412.46,0.00,157.42,0.0,0.0315,0.3760,...,-0.00058,0.00001,0.00000,1.00000,2537.49,0.1,0.0,0.3770,0.3770,month
0,0.0,0.0,BTC-25OCT24,67154.81,0.00,0.00,171.48,0.0,0.0000,0.0000,...,-0.00014,0.00000,0.00000,1.00000,0.00,0.0,NaN,NaN,NaN,month
0,0.0,0.1,BTC-25OCT24,67154.81,489.40,0.00,171.48,0.0,0.0000,0.0001,...,0.00000,0.00000,0.00000,0.00000,0.00,0.0,NaN,NaN,NaN,month
0,0.0,0.1,BTC-25OCT24,67154.81,297.11,0.00,157.42,0.0,0.0000,0.0001,...,-0.00005,0.00001,0.00000,0.00000,0.00,0.0,NaN,NaN,NaN,month
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,0.0,0.0,BTC-26SEP25,64817.85,0.00,0.00,62.42,0.0,0.0000,0.0000,...,-12.59085,136.31606,0.00001,-0.86367,0.00,0.0,NaN,NaN,NaN,month
0,0.0,0.0,BTC-26SEP25,64817.85,0.00,0.00,63.56,0.0,0.0000,0.0000,...,-10.43673,110.95298,0.00000,-0.89811,0.00,0.0,NaN,NaN,NaN,month
0,5.7,5.5,BTC-26SEP25,64817.85,69.51,54.69,62.92,0.0,1.4825,1.5095,...,-11.39717,122.14127,0.00001,-0.88336,0.00,0.0,NaN,NaN,NaN,month
0,0.0,0.0,BTC-26SEP25,64817.85,0.00,0.00,64.56,0.0,0.0000,0.0000,...,-8.64915,90.37319,0.00000,0.07744,14541.21,12.5,0.0,0.0173,0.0173,month


In [21]:
# Extract Expiration date and strike price

import re
# Define a function to extract expiration date and strike price
def extract_details(instrument_name):
    match = re.match(r"BTC-(\d+[A-Z]{3}\d+)-(\d+)-[CP]", instrument_name)
    if match:
        expiration_date = match.group(1)
        strike_price = match.group(2)
        return expiration_date, strike_price
    return None, None

# Apply the function to your data
btc_data['Expiration Date'], btc_data['Strike Price'] = zip(*btc_data['instrument_name'].apply(extract_details))

btc_data.to_csv('data/btc_data.csv', index=False)
#eth_data.to_csv('data/eth_data.csv', index=False)
